# CH07-07: 大型語言模型 (Large Language Models)

**課程時長**: 90 分鐘  
**難度**: ⭐⭐⭐⭐  
**前置知識**: CH07-01 至 CH07-06  

---

## 📚 本節學習目標

1. ✅ 理解 LLM 的定義與規模定律 (Scaling Laws)
2. ✅ 掌握 Instruction Tuning 與 RLHF 原理
3. ✅ 學會 Prompt Engineering 核心技巧
4. ✅ 理解 In-Context Learning 的工作機制
5. ✅ 認識 LLM 的能力邊界與倫理議題

---

## 📖 目錄

1. [什麼是大型語言模型](#1-what-is-llm)
2. [規模定律與湧現能力](#2-scaling-laws)
3. [從預訓練到對齊](#3-alignment)
4. [Prompt Engineering 實戰](#4-prompt-engineering)
5. [In-Context Learning 原理](#5-in-context-learning)
6. [LLM 的局限性與未來](#6-limitations)
7. [總結](#7-summary)

---

## 1. 什麼是大型語言模型 {#1-what-is-llm}

### 1.1 定義

**Large Language Model (LLM)**:
> 擁有數十億至數兆參數的 Transformer 模型，在大規模文本語料上預訓練，展現出強大的語言理解與生成能力。

**關鍵特徵**:
- **規模**: 參數量通常 > 1B (10億)
- **架構**: 主要基於 Decoder-Only Transformer
- **訓練數據**: 數百 GB 至 TB 級文本
- **能力**: Few-shot learning, In-context learning, Chain-of-thought reasoning

### 1.2 LLM 時間軸

```
2018 ─ GPT (117M)          └─ 證明預訓練有效性
       BERT (340M)

2019 ─ GPT-2 (1.5B)        └─ 首個 10億+ 參數模型
       T5 (11B)

2020 ─ GPT-3 (175B)        └─ Few-shot learning 突破

2021 ─ Codex (12B)         └─ 專精程式碼
       DALL-E (12B)

2022 ─ ChatGPT             └─ RLHF + Instruction Tuning
       InstructGPT

2023 ─ GPT-4 (~1.8T?)      └─ Multimodal LLM
       LLaMA (7B-65B)      └─ 開源高效 LLM
       Mistral (7B)        └─ MoE 架構

2024 ─ Gemini 1.5 Pro      └─ 超長上下文 (1M tokens)
       Claude 3            └─ 優秀推理能力
```

### 1.3 參數規模對比

In [ ]:
# 載入必要套件
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from typing import List, Dict
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 套件載入完成")

In [ ]:
# 視覺化模型參數規模演進

models_data = {
    'Model': [
        'BERT-Large', 'GPT', 'GPT-2', 'T5-11B', 'GPT-3', 
        'LLaMA-65B', 'PaLM', 'GPT-4 (估計)'
    ],
    'Parameters (B)': [0.34, 0.117, 1.5, 11, 175, 65, 540, 1800],
    'Year': [2018, 2018, 2019, 2019, 2020, 2023, 2022, 2023],
    'Type': ['Encoder', 'Decoder', 'Decoder', 'Enc-Dec', 'Decoder', 
             'Decoder', 'Decoder', 'Decoder']
}

df = pd.DataFrame(models_data)

# 繪圖
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左圖: 對數尺度的參數增長
colors = {'Encoder': 'skyblue', 'Decoder': 'lightcoral', 'Enc-Dec': 'lightgreen'}
for model_type in df['Type'].unique():
    subset = df[df['Type'] == model_type]
    axes[0].scatter(
        subset['Year'], 
        subset['Parameters (B)'], 
        s=200, 
        alpha=0.7,
        label=model_type,
        color=colors[model_type]
    )
    
    for _, row in subset.iterrows():
        axes[0].text(
            row['Year'], 
            row['Parameters (B)'] * 1.3, 
            row['Model'], 
            ha='center', 
            fontsize=9
        )

axes[0].set_yscale('log')
axes[0].set_xlabel('發布年份', fontsize=12)
axes[0].set_ylabel('參數量 (Billions, 對數尺度)', fontsize=12)
axes[0].set_title('LLM 參數規模演進 (2018-2023)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 右圖: 參數量條形圖
df_sorted = df.sort_values('Parameters (B)')
bar_colors = [colors[t] for t in df_sorted['Type']]

axes[1].barh(df_sorted['Model'], df_sorted['Parameters (B)'], color=bar_colors, alpha=0.7)
axes[1].set_xlabel('參數量 (Billions)', fontsize=12)
axes[1].set_title('模型參數量對比', fontsize=14, fontweight='bold')
axes[1].set_xscale('log')
axes[1].grid(axis='x', alpha=0.3)

for i, (model, params) in enumerate(zip(df_sorted['Model'], df_sorted['Parameters (B)'])):
    axes[1].text(params * 1.2, i, f'{params}B', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n關鍵觀察:")
print(f"• 2018-2023 年間，最大模型參數增長了 {1800/0.117:.0f}x 倍")
print("• Decoder-Only 架構主導 LLM 發展")
print("• GPT-4 參數量約為 GPT-3 的 10 倍")

## 2. 規模定律與湧現能力 {#2-scaling-laws}

### 2.1 Scaling Laws (規模定律)

**核心發現** (Kaplan et al., 2020):
> 語言模型的效能與三個因素呈現冪律關係 (Power Law):
> 1. 模型參數量 (N)
> 2. 訓練數據量 (D)
> 3. 計算量 (C)

**數學形式**:
$$
\text{Loss}(N) \propto N^{-\alpha}
$$

**實際意義**:
- 10x 參數 → 顯著的效能提升
- 但效能提升速度遞減 (邊際效應)

### 2.2 湧現能力 (Emergent Abilities)

**定義**:
> 當模型規模超過某個臨界點時，突然展現出小模型不具備的能力。

**典型湧現能力**:

1. **算術推理**
   - < 10B: 幾乎無法解決
   - 10B-100B: 開始出現能力
   - > 100B: 接近人類水平

2. **多步推理 (Chain-of-Thought)**
   ```
   問題: "Roger 有 5 顆網球。他又買了 2 罐網球，每罐 3 顆。他現在有幾顆網球?"
   
   小模型 (<10B): "7 顆" ❌
   
   大模型 (>100B):
   "讓我們一步步思考:
   1. Roger 原本有 5 顆網球
   2. 他買了 2 罐，每罐 3 顆，所以買了 2 × 3 = 6 顆
   3. 總共有 5 + 6 = 11 顆網球" ✅
   ```

3. **Few-Shot Learning**
   - 從少量範例中學習新任務
   - 無需梯度更新

4. **Code Generation**
   - 從自然語言生成可執行程式碼

### 2.3 視覺化湧現能力

In [ ]:
# 模擬湧現能力曲線

def sigmoid_emergence(x, threshold, steepness):
    """模擬湧現能力的 sigmoid 曲線"""
    return 1 / (1 + np.exp(-steepness * (x - threshold)))

# 參數規模 (對數尺度)
params = np.logspace(8, 12, 100)  # 100M to 1T

# 不同能力的湧現點
abilities = {
    '基礎語言能力': (9.5, 2),
    'Few-Shot Learning': (10.5, 3),
    '算術推理': (11.0, 4),
    'Chain-of-Thought': (11.5, 5),
    '複雜推理': (11.8, 6)
}

fig, ax = plt.subplots(figsize=(14, 8))

for ability, (threshold, steepness) in abilities.items():
    performance = sigmoid_emergence(np.log10(params), threshold, steepness)
    ax.plot(params, performance * 100, label=ability, linewidth=2.5)

ax.set_xscale('log')
ax.set_xlabel('模型參數量', fontsize=13)
ax.set_ylabel('任務表現 (%)', fontsize=13)
ax.set_title('LLM 湧現能力: 不同規模下的表現', fontsize=15, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

# 標記重要里程碑
milestones = [
    (1e9, 'GPT-2\n1.5B'),
    (1e11, 'GPT-3\n175B'),
    (5e11, 'PaLM\n540B')
]

for param, label in milestones:
    ax.axvline(param, color='gray', linestyle='--', alpha=0.5)
    ax.text(param, 95, label, ha='center', fontsize=9, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.set_xticks([1e8, 1e9, 1e10, 1e11, 1e12])
ax.set_xticklabels(['100M', '1B', '10B', '100B', '1T'])

plt.tight_layout()
plt.show()

print("\n湧現能力的特點:")
print("• 非線性: 能力突然爆發，而非平滑增長")
print("• 閾值效應: 存在明確的臨界規模")
print("• 不可預測: 難以預先知道何時會湧現新能力")

## 3. 從預訓練到對齊 {#3-alignment}

### 3.1 LLM 訓練的三個階段

```
階段 1: 預訓練 (Pre-training)
        ↓
   學到語言結構與知識
        ↓
階段 2: 指令微調 (Instruction Tuning)
        ↓
   學會遵循指令
        ↓
階段 3: 人類反饋強化學習 (RLHF)
        ↓
   對齊人類價值觀
```

### 3.2 階段 1: 預訓練 (Pre-training)

**目標**: 學習語言的統計結構與世界知識

**訓練任務**: Causal Language Modeling
```python
Input:  "The capital of France is"
Target: "Paris"

Loss = -log P("Paris" | "The capital of France is")
```

**訓練數據**:
- Common Crawl (網頁)
- Books (書籍)
- Wikipedia (百科)
- GitHub (程式碼)
- Reddit, StackOverflow (論壇)

**GPT-3 數據組成**:
- Common Crawl: 60%
- WebText2: 22%
- Books: 16%
- Wikipedia: 3%

**挑戰**:
- 計算成本極高 (數百萬美元)
- 需要數千個 GPU/TPU
- 訓練時間: 數週至數月

### 3.3 階段 2: 指令微調 (Instruction Tuning)

**目標**: 讓模型理解並遵循人類指令

**資料格式**:
```python
# 範例 1: 問答
Instruction: "What is the capital of France?"
Response: "The capital of France is Paris."

# 範例 2: 翻譯
Instruction: "Translate to French: Hello, how are you?"
Response: "Bonjour, comment allez-vous?"

# 範例 3: 程式碼
Instruction: "Write a Python function to calculate factorial"
Response: "def factorial(n):\n    if n == 0: return 1\n    return n * factorial(n-1)"
```

**資料集**:
- **FLAN** (Google): 1800+ 任務
- **T0** (BigScience): 多樣化 prompts
- **Self-Instruct** (Stanford): 使用 GPT-3 生成指令數據

**效果**:
- 顯著提升遵循指令的能力
- 改善 zero-shot 表現
- 減少有害輸出

### 3.4 階段 3: RLHF (Reinforcement Learning from Human Feedback)

**核心思想**: 用人類偏好訓練獎勵模型，再用 RL 優化 LLM

**流程**:

```
Step 1: 收集人類偏好數據
   Prompt: "寫一首關於春天的詩"
   Response A: [詩歌 A]
   Response B: [詩歌 B]
   人類標註: A > B  (A 更好)

Step 2: 訓練獎勵模型 (Reward Model)
   輸入: (Prompt, Response)
   輸出: Reward Score (預測人類滿意度)

Step 3: 用 PPO 演算法優化 LLM
   目標: 最大化獎勵模型的分數
   約束: 不要偏離原始模型太遠 (KL divergence penalty)
```

**數學形式**:
$$
\text{Objective} = \mathbb{E}_{x \sim D, y \sim \pi_\theta}[r(x, y)] - \beta \cdot D_{KL}(\pi_\theta || \pi_{\text{ref}})
$$

其中:
- $r(x, y)$: 獎勵模型分數
- $\pi_\theta$: 當前策略 (LLM)
- $\pi_{\text{ref}}$: 參考策略 (原始模型)
- $\beta$: KL 懲罰係數

In [ ]:
# 視覺化 RLHF 流程

fig, ax = plt.subplots(figsize=(14, 10))

# 定義流程圖節點
steps = [
    (0.5, 0.9, "預訓練 LLM", 'lightblue'),
    (0.5, 0.75, "Instruction Tuning", 'lightgreen'),
    (0.2, 0.55, "收集偏好數據\n(A vs B)", 'lightyellow'),
    (0.5, 0.55, "訓練獎勵模型\n(Reward Model)", 'lightcoral'),
    (0.8, 0.55, "生成多個回應", 'lavender'),
    (0.5, 0.35, "PPO 強化學習優化", 'lightgreen'),
    (0.5, 0.15, "對齊後的 LLM\n(ChatGPT)", 'gold')
]

for x, y, label, color in steps:
    ax.add_patch(plt.Rectangle(
        (x-0.12, y-0.05), 0.24, 0.08, 
        facecolor=color, edgecolor='black', linewidth=2
    ))
    ax.text(x, y, label, ha='center', va='center', fontsize=11, fontweight='bold')

# 繪製箭頭
arrows = [
    ((0.5, 0.82), (0.5, 0.75)),
    ((0.5, 0.70), (0.2, 0.60)),
    ((0.5, 0.70), (0.5, 0.60)),
    ((0.5, 0.70), (0.8, 0.60)),
    ((0.2, 0.50), (0.5, 0.40)),
    ((0.5, 0.50), (0.5, 0.40)),
    ((0.8, 0.50), (0.5, 0.40)),
    ((0.5, 0.30), (0.5, 0.20))
]

for start, end in arrows:
    ax.annotate('', xy=end, xytext=start,
               arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))

# 添加說明文字
annotations = [
    (0.15, 0.60, "人類評分員\n比較回應", 9),
    (0.65, 0.60, "LLM 生成\n候選回應", 9),
    (0.75, 0.35, "最大化\n獎勵分數", 9, 'green'),
    (0.25, 0.35, "學習\n人類偏好", 9, 'red')
]

for x, y, text, size, *color in annotations:
    c = color[0] if color else 'black'
    ax.text(x, y, text, ha='center', fontsize=size, style='italic', color=c,
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('RLHF 完整流程: 從預訓練到對齊', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\nRLHF 的重要性:")
print("• 使 LLM 更安全、更有用、更誠實")
print("• 減少有害、偏見、錯誤的輸出")
print("• ChatGPT 的成功關鍵之一")

## 4. Prompt Engineering 實戰 {#4-prompt-engineering}

### 4.1 什麼是 Prompt Engineering?

**定義**:
> 設計和優化輸入文本 (prompt)，以引導 LLM 產生期望的輸出。

**為什麼重要?**
- LLM 對 prompt 措辭極度敏感
- 好的 prompt 可以大幅提升表現
- 無需微調，成本低廉

### 4.2 核心技巧

#### 技巧 1: Zero-Shot Prompting

**基礎版**:
```
Prompt: "Classify the sentiment: I love this product!"
Output: "Positive"
```

**改進版** (更明確的指令):
```
Prompt: "Classify the sentiment of the following text as Positive, Negative, or Neutral.\n\nText: I love this product!\nSentiment:"
Output: "Positive"
```

#### 技巧 2: Few-Shot Prompting

提供範例來引導模型:

```
Classify sentiment as Positive, Negative, or Neutral.

Text: "This movie was amazing!"
Sentiment: Positive

Text: "I hated this book."
Sentiment: Negative

Text: "The weather is okay."
Sentiment: Neutral

Text: "Best purchase ever!"
Sentiment:
```

#### 技巧 3: Chain-of-Thought (CoT) Prompting

**標準 Prompt** (表現差):
```
Q: Roger 有 5 顆網球。他又買了 2 罐，每罐 3 顆。他現在有幾顆?
A: 7
```

**CoT Prompt** (表現好):
```
Q: Roger 有 5 顆網球。他又買了 2 罐，每罐 3 顆。他現在有幾顆?
A: Let's think step by step.
1. Roger 原本有 5 顆網球
2. 他買了 2 罐，每罐 3 顆，所以是 2 × 3 = 6 顆
3. 總共 5 + 6 = 11 顆
答案是 11 顆。
```

**神奇的一句話**: `"Let's think step by step."`

#### 技巧 4: Self-Consistency

生成多個推理路徑，取多數決:

```
同一問題生成 5 次 (temperature > 0):
路徑 1: ... 答案是 11
路徑 2: ... 答案是 11
路徑 3: ... 答案是 7  (錯誤)
路徑 4: ... 答案是 11
路徑 5: ... 答案是 11

最終答案: 11 (多數決)
```

#### 技巧 5: Role Prompting

```
差: "寫一篇關於氣候變遷的文章"

好: "你是一位專精氣候科學的資深教授。請用淺顯易懂的方式，向高中生解釋氣候變遷的原因與影響。"
```

### 4.3 Prompt 設計原則

In [ ]:
# 展示 Prompt Engineering 的最佳實踐

principles = {
    '原則': [
        '1. 明確性',
        '2. 具體性',
        '3. 範例導向',
        '4. 結構化輸出',
        '5. 迭代優化'
    ],
    '差的 Prompt': [
        '"分類這個"',
        '"寫點東西"',
        '沒有範例',
        '"告訴我結果"',
        '一次就想完美'
    ],
    '好的 Prompt': [
        '"將以下文本分類為 Positive/Negative"',
        '"用 3 段落介紹 Transformer，每段 100 字"',
        '提供 2-3 個範例',
        '"以 JSON 格式輸出: {\"sentiment\": ...}"',
        '測試多個版本，選最佳'
    ]
}

df_principles = pd.DataFrame(principles)

# 使用 HTML 顯示表格
print("=" * 80)
print("Prompt Engineering 設計原則")
print("=" * 80)
print()

for _, row in df_principles.iterrows():
    print(f"{row['原則']:20s}")
    print(f"  ❌ 差: {row['差的 Prompt']}")
    print(f"  ✅ 好: {row['好的 Prompt']}")
    print()

print("=" * 80)
print("\n核心建議:")
print("• 把 LLM 當作一個聰明但字面意義的助手")
print("• 提供上下文、範例、格式要求")
print("• 迭代測試，記錄有效的 prompts")
print("• 使用 temperature=0 保證穩定輸出")

## 5. In-Context Learning 原理 {#5-in-context-learning}

### 5.1 什麼是 In-Context Learning?

**定義**:
> LLM 能夠在推論時從 prompt 中的範例學習，執行新任務，無需更新參數。

**神奇之處**:
- 不需要梯度下降
- 不需要微調
- 只需要在 prompt 中提供範例

**範例**:
```python
# 模型從未見過這個任務，但能立即學會
Prompt = """
將句子轉換為過去式:

I walk to school → I walked to school
She eats an apple → She ate an apple
They play soccer → They played soccer
He runs fast →
"""

Output: "He ran fast"
```

### 5.2 工作機制假說

**假說 1: 隱式微調**
- LLM 內部執行了類似梯度下降的過程
- 前向傳播中調整內部表示

**假說 2: 模式匹配**
- 訓練時見過類似的 few-shot 模式
- 識別並複製該模式

**假說 3: 貝葉斯推斷**
- LLM 維護多個假說
- 根據範例更新假說的後驗機率

**最新研究**:
- Transformer 可以實現梯度下降 (Von Oswald et al., 2023)
- In-context learning ≈ 隱式參數更新

### 5.3 影響 In-Context Learning 的因素

**1. 範例數量**:
- 0-shot < 1-shot < few-shot (通常 3-5 個最佳)
- 但超過某個數量後邊際效益遞減

**2. 範例品質**:
- 多樣性 > 相似性
- 代表性範例 > 極端案例

**3. 範例順序**:
- 重要! 順序會影響結果
- 建議: 隨機化測試多種順序

**4. 模型規模**:
- < 10B: In-context learning 能力弱
- > 100B: 顯著提升

In [ ]:
# 模擬 In-Context Learning 效能與範例數量的關係

def icl_performance(num_examples, model_size):
    """模擬 ICL 表現隨範例數增長"""
    # 使用對數函數模擬邊際遞減
    base_performance = 30 * (model_size / 100)  # 模型規模基準
    improvement = 30 * np.log1p(num_examples) * (model_size / 100)
    noise = np.random.normal(0, 2)
    return min(95, base_performance + improvement + noise)

num_examples_range = range(0, 11)
model_sizes = [10, 100, 500]  # 10B, 100B, 500B

fig, ax = plt.subplots(figsize=(12, 7))

for size in model_sizes:
    performances = [icl_performance(n, size) for n in num_examples_range]
    ax.plot(num_examples_range, performances, marker='o', linewidth=2.5, 
            label=f'{size}B 參數', markersize=8)

ax.set_xlabel('範例數量 (Few-Shot)', fontsize=13)
ax.set_ylabel('任務準確率 (%)', fontsize=13)
ax.set_title('In-Context Learning: 範例數量 vs 表現', fontsize=15, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xticks(range(0, 11))

# 標註關鍵區域
ax.axvspan(-0.5, 0.5, alpha=0.1, color='red', label='Zero-Shot')
ax.axvspan(0.5, 1.5, alpha=0.1, color='orange', label='One-Shot')
ax.axvspan(1.5, 10.5, alpha=0.1, color='green', label='Few-Shot')

ax.text(0, 25, 'Zero-Shot', ha='center', fontsize=10, style='italic')
ax.text(1, 25, 'One-Shot', ha='center', fontsize=10, style='italic')
ax.text(5, 25, 'Few-Shot', ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.show()

print("\n關鍵觀察:")
print("• 大模型從 0 個範例就有不錯表現")
print("• 3-5 個範例通常是最佳平衡點")
print("• 小模型 (<10B) 幾乎無 in-context learning 能力")

## 6. LLM 的局限性與未來 {#6-limitations}

### 6.1 核心局限

#### 1. 幻覺 (Hallucination)

**定義**: 生成聽起來合理但實際錯誤的內容

**範例**:
```
Q: "誰發明了電燈泡?"
A: "湯瑪斯·愛迪生於 1879 年發明了電燈泡。" ✅ 正確

Q: "愛迪生在哪所大學獲得博士學位?"
A: "愛迪生在耶魯大學獲得物理學博士學位。" ❌ 編造 (他沒有大學學位)
```

**原因**:
- 訓練目標是預測下一個詞，而非保證真實性
- 知識截止日期後的資訊無法獲得
- 訓練數據可能包含錯誤資訊

**緩解方法**:
- 檢索增強生成 (RAG)
- 引用來源
- 人工審核關鍵輸出

#### 2. 上下文長度限制

| 模型 | 上下文長度 | 可處理文本量 |
|------|-----------|-------------|
| GPT-3 | 2,048 tokens | ~1,500 英文單字 |
| GPT-3.5 | 4,096 tokens | ~3,000 英文單字 |
| GPT-4 | 8,192 / 32,768 | ~6,000 / 24,000 單字 |
| Claude 3 | 200,000 tokens | ~150,000 單字 |
| Gemini 1.5 | 1,000,000 tokens | ~750,000 單字 |

**挑戰**:
- 無法處理超長文檔
- 計算複雜度 O(n²)

#### 3. 推理能力限制

**擅長**:
- 模式識別
- 簡單推理
- 常識應用

**不擅長**:
- 複雜數學證明
- 多步邏輯推理
- 需要精確計算的任務

**範例 (失敗案例)**:
```
Q: "計算 4,567 × 8,923"
A: "40,759,241"  ❌ (正確答案: 40,755,241)
```

#### 4. 訓練成本與環境影響

**GPT-3 訓練成本估計**:
- 電力: ~1,287 MWh
- 碳排放: ~552 噸 CO₂
- 金錢: $4.6M USD

#### 5. 偏見與安全性

**問題來源**:
- 訓練數據包含社會偏見
- 可能生成有害內容
- 隱私洩露風險

**應對措施**:
- RLHF 對齊
- 內容過濾
- 持續監控與改進

### 6.2 未來趨勢

**1. 多模態 LLM**
- GPT-4, Gemini: 文本 + 圖像
- 未來: 音訊 + 視頻 + 3D

**2. 效率提升**
- Mixture of Experts (MoE)
- 量化 (Quantization)
- 稀疏激活

**3. 更長上下文**
- 突破 1M tokens
- 高效注意力機制

**4. 工具使用 (Tool Use)**
- 呼叫外部 API
- 執行程式碼
- 檢索即時資訊

**5. 多智能體系統**
- 多個 LLM 協作
- 專精不同領域
- 自我修正與辯論

In [ ]:
# 視覺化 LLM 發展趨勢

trends = {
    '趨勢': [
        '模型規模',
        '上下文長度',
        '多模態能力',
        '推理能力',
        '效率 (成本)',
        '安全性對齊'
    ],
    '2023': [85, 60, 50, 55, 40, 65],
    '2024 (預測)': [90, 80, 75, 70, 60, 80],
    '2025 (預測)': [92, 90, 85, 80, 75, 90]
}

df_trends = pd.DataFrame(trends)
df_trends = df_trends.set_index('趨勢')

# 繪製雷達圖
categories = df_trends.index.tolist()
N = len(categories)

angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for year in df_trends.columns:
    values = df_trends[year].tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=year, markersize=8)
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=11)
ax.set_ylim(0, 100)
ax.set_yticks([20, 40, 60, 80, 100])
ax.set_yticklabels(['20', '40', '60', '80', '100'], fontsize=9)
ax.set_title('LLM 能力發展趨勢 (雷達圖)', fontsize=15, fontweight='bold', pad=30)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
ax.grid(True)

plt.tight_layout()
plt.show()

print("\n趨勢解讀:")
print("• 上下文長度: 快速增長 (1M+ tokens 成為可能)")
print("• 多模態: 從單純文本擴展到圖像、音訊、視頻")
print("• 效率: 透過 MoE, 量化等技術大幅降低成本")
print("• 安全性: 更強的對齊與過濾機制")

## 7. 總結 {#7-summary}

### 7.1 關鍵要點回顧

✅ **LLM 的核心**:
- 定義: > 1B 參數的 Transformer 模型
- 架構: 主要為 Decoder-Only
- 能力: Few-shot learning, Chain-of-thought, In-context learning

✅ **規模定律**:
- 效能 ∝ 參數量^(-α)
- 湧現能力在特定規模突然出現
- 但邊際效應遞減

✅ **訓練三階段**:
1. 預訓練: 學習語言與知識
2. Instruction Tuning: 學會遵循指令
3. RLHF: 對齊人類價值觀

✅ **Prompt Engineering**:
- 明確、具體、結構化
- Few-shot > One-shot > Zero-shot
- Chain-of-Thought 提升推理
- 迭代優化是關鍵

✅ **In-Context Learning**:
- 無需梯度更新的學習
- 3-5 個範例最佳
- 需要 > 10B 參數才有效

✅ **局限性**:
- 幻覺 (編造事實)
- 上下文長度限制
- 推理能力有限
- 訓練成本高
- 偏見與安全問題

### 7.2 實用建議

**使用 LLM 的最佳實踐**:

1. **選擇合適模型**:
   - 簡單任務: GPT-3.5, Claude Instant
   - 複雜任務: GPT-4, Claude 3
   - 預算有限: 開源 LLaMA, Mistral

2. **設計有效 Prompt**:
   - 從簡單開始，逐步優化
   - 記錄有效的 prompts
   - 使用 prompt 模板庫

3. **處理輸出**:
   - 驗證關鍵事實
   - 使用多次生成 + 投票
   - 結合檢索系統 (RAG)

4. **成本控制**:
   - 快取常用結果
   - 使用較小模型處理簡單任務
   - 優化 prompt 長度

### 7.3 延伸閱讀

**論文**:
1. [Language Models are Few-Shot Learners (GPT-3)](https://arxiv.org/abs/2005.14165)
2. [Training language models to follow instructions (InstructGPT)](https://arxiv.org/abs/2203.02155)
3. [Chain-of-Thought Prompting](https://arxiv.org/abs/2201.11903)
4. [Scaling Laws for Neural Language Models](https://arxiv.org/abs/2001.08361)

**資源**:
- [OpenAI Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Awesome LLM (GitHub)](https://github.com/Hannibal046/Awesome-LLM)

---

## 🎯 下一節預告

**CH07-08: LLM 實際應用案例**
- RAG (檢索增強生成) 實作
- LangChain 框架使用
- Agent 系統設計
- 使用 Hugging Face API 呼叫開源 LLM
- 生產環境部署考量

---

**課程完成時間**: `____年____月____日`  
**學習心得**: ___________________________________